In [ ]:
import pandas as pd

#初始文件，修改索引列,格式转换
df1 = pd.read_csv('/content/Conley2 - 副本.csv',encoding = 'GBK')
# 将登记时间列转换为日期类型
df1["Latest 中登网 Date (风暴)"] = pd.to_datetime(df1["Latest 中登网 Date (风暴)"])
# 将日期类型的列转换为指定格式的字符串
df1["new_time"] = df1["Latest 中登网 Date (风暴)"].dt.strftime("%Y-%-m-%-d")
# # 输出结果
# 去除空行
df1 = df1.dropna(how='all')

# print(df1["new_time"])

#目标文件
df = pd.read_excel("/content/登记详细信息列表 (4).xls", sheet_name="融资租赁")
df["登记时间"] = pd.to_datetime(df["登记时间"])
df["time_new"] = df["登记时间"].dt.strftime("%Y-%-m-%-d")
# print(df["time_new"])

Test

In [ ]:
import datetime

merged_df = pd.merge(df, df1, left_on="承租人名称（金融机构、企业、机关事业单位、其他）/\n字号名称（个体工商户）", right_on="Chinese Company Name 公司中文名字", how="inner")
print(merged_df["Chinese Company Name 公司中文名字"][0])

# 定义 datetime2
datetime2 = pd.to_datetime(merged_df["new_time"])

registration_types = []
time_new_values = []

# 根据"初始登记"值将"登记类型"列分割成多个列表
grouped_types = []
grouped_time_new = []
for i, row in merged_df.iterrows():
    reg_type = row["登记类型"]
    time_new = row["time_new"]
    if reg_type == "初始登记":
        if grouped_types:
            registration_types.append(grouped_types)
            time_new_values.append(grouped_time_new)
            grouped_types = []
            grouped_time_new = []
        grouped_types.append(reg_type)
        grouped_time_new.append([time_new])  # 将单个值包装在列表中
    else:
        grouped_types.append(reg_type)
        grouped_time_new[-1].append(time_new)  # 添加到最后一个列表中

# 添加最后一个列表
registration_types.append(grouped_types)
time_new_values.append(grouped_time_new)

# 将"登记类型"和"time_new"组合成字典
result = {}
for types, times in zip(registration_types, time_new_values):
    if len(types) == 1 and types[0] == "初始登记":
        if types[0] in result:
            result[types[0]].extend(times)
        else:
            result[types[0]] = times
    else:
        key = tuple(types)
        if key in result:
            result[key].extend(times)
        else:
            result[key] = times


# 输出结果
result_dict = {}
for key, value in result.items():
    length = len(key)
    if length in result_dict:
        result_dict[length][key] = value
    else:
        result_dict[length] = {key: value}

import pandas as pd

df_first = pd.DataFrame(columns=["Form Filler", "Leasing Time", "Leasing Value"])
df_last = pd.DataFrame(columns=["Form Filler", "Leasing Time", "Leasing Value"])

for length, result in result_dict.items():
    if length == 4:
        # print(f"字典长度为 {length}")

        for key, values in result.items():
            if '初始登记' in key:
                # print(key)
                converted_values = []
                for sublist in values:
                    converted_sublist = [datetime.datetime.strptime(date_str, '%Y-%m-%d') for date_str in sublist]
                    converted_values.append(converted_sublist)

                # 遍历转换后的值
                for sublist in converted_values:
                    for date in sublist:
                        # 遍历 datetime2 的每个元素进行比较
                        found_match = False
                        for dt in datetime2:
                            if date > dt:
                                date1 = date.strftime('%Y-%m-%d')
                                print("Leasing Time:", date1)
                                found_match = True
                                break  # 找到匹配后立即终止内部循环

                        if found_match:
                            merged_df["time_new"] = pd.to_datetime(merged_df["time_new"])
                            date1_str = pd.to_datetime(date1).strftime('%Y-%m-%d')
                            filtered_row = merged_df[merged_df["time_new"].dt.strftime('%Y-%m-%d').str.match(date1_str)]

                            if not filtered_row.empty:
                                leasing_value = filtered_row["租赁财产价值"].values[0]
                                leasing_value /= 1000  # 将 leasing_value 的值除以 1000
                                form_filler = filtered_row["填表人"].values[0]
                                # print("Leasing Value:", leasing_value)
                                # print("Form Filler:", form_filler)

                                df_first = df_first.append(
                                    {"Form Filler": form_filler, "Leasing Time": date1, "Leasing Value": leasing_value},
                                    ignore_index=True
                                )

    else:
        for key, values in result.items():
            if "注销融资" not in key:
                # print(key)
                converted_values = []
                for sublist in values:
                    converted_sublist = [datetime.datetime.strptime(date_str, '%Y-%m-%d') for date_str in sublist]
                    last_value = converted_sublist[-1]  # 获取子列表中的最后一个数值
                    first_value = converted_sublist[0]
                    converted_values.append([last_value])  # 将最后一个数值作为单个元素的列表添加到新列表中

                # 遍历转换后的值
                for sublist in converted_values:
                    for date_value in sublist:
                        # 遍历 datetime2 的每个元素进行比较
                        found_match = False
                        for dt in datetime2:
                            if date_value > dt:
                                date3 = date_value.strftime('%Y-%m-%d')
                                found_match = True
                                break  # 找到匹配后立即终止内部循环
                        if found_match:
                            merged_df["time_new"] = pd.to_datetime(merged_df["time_new"])
                            date3_str = pd.to_datetime(date3).strftime('%Y-%m-%d')
                            first_date = datetime.datetime.strptime(values[0][0], '%Y-%m-%d')
                            date4_str = first_date.strftime('%Y-%m-%d')
                            # print("Leasing Time:", date4_str)
                            filtered_row1 = merged_df[merged_df["time_new"].dt.strftime('%Y-%m-%d').str.match(date3_str)]
                            filtered_row2 = merged_df[merged_df["time_new"].dt.strftime('%Y-%m-%d').str.match(date4_str)]
                            if not filtered_row1.empty and not filtered_row2.empty:
                                leasing_value = filtered_row1["租赁财产价值"].values[0]
                                leasing_value /= 1000  # 将 leasing_value 的值除以 1000
                                form_filler = filtered_row1["填表人"].values[0]
                                # print("Leasing Value:", leasing_value)
                                # print("Form Filler:", form_filler)

                                df_last = df_last.append(
                                    {"Form Filler": form_filler, "Leasing Time": date4_str, "Leasing Value": leasing_value},
                                    ignore_index=True
                                )

# 调整列顺序
df_first = df_first[["Form Filler", "Leasing Time", "Leasing Value"]]
df_last = df_last[["Form Filler", "Leasing Time", "Leasing Value"]]

print("df_first:")
print(df_first)
print("df_last:")
print(df_last)

Final

In [ ]:
import datetime

merged_df = pd.merge(df, df1, left_on="承租人名称（金融机构、企业、机关事业单位、其他）/\n字号名称（个体工商户）", right_on="Chinese Company Name 公司中文名字", how="inner")
print(merged_df["Chinese Company Name 公司中文名字"][0])

datetime2 = pd.to_datetime(merged_df["new_time"])

registration_types = []
time_new_values = []

# 根据"初始登记"值将"登记类型"列分割成多个列表
grouped_types = []
grouped_time_new = []
for i, row in merged_df.iterrows():
    reg_type = row["登记类型"]
    time_new = row["time_new"]
    if reg_type == "初始登记":
        if grouped_types:
            registration_types.append(grouped_types)
            time_new_values.append(grouped_time_new)
            grouped_types = []
            grouped_time_new = []
        grouped_types.append(reg_type)
        grouped_time_new.append([time_new])  # 将单个值包装在列表中
    else:
        grouped_types.append(reg_type)
        grouped_time_new[-1].append(time_new)  # 添加到最后一个列表中

# 添加最后一个列表
registration_types.append(grouped_types)
time_new_values.append(grouped_time_new)

# 将"登记类型"和"time_new"组合成字典
result = {}
for types, times in zip(registration_types, time_new_values):
    if len(types) == 1 and types[0] == "初始登记":
        if types[0] in result:
            result[types[0]].extend(times)
        else:
            result[types[0]] = times
    else:
        key = tuple(types)
        if key in result:
            result[key].extend(times)
        else:
            result[key] = times

# 输出结果
result_dict = {}
for key, value in result.items():
    length = len(key)
    if length in result_dict:
        result_dict[length][key] = value
    else:
        result_dict[length] = {key: value}


df_first = pd.DataFrame(columns=["填表人", "租赁时间", "租赁价值"])
df_last = pd.DataFrame(columns=["填表人", "租赁时间", "租赁价值"])

for length, result in result_dict.items():
    if length == 4:
        for key, values in result.items():
            if '初始登记' in key:
                converted_values = []
                for sublist in values:
                    converted_sublist = [datetime.datetime.strptime(date_str, '%Y-%m-%d') for date_str in sublist]
                    converted_values.append(converted_sublist)

                for sublist in converted_values:
                    for date in sublist:
                        found_match = False
                        for dt in datetime2:
                            if date > dt:
                                date1 = date.strftime('%Y-%m-%d')
                                found_match = True
                                break

                        if found_match:
                            merged_df["time_new"] = pd.to_datetime(merged_df["time_new"])
                            date1_str = pd.to_datetime(date1).strftime('%Y-%m-%d')
                            filtered_row = merged_df[merged_df["time_new"].dt.strftime('%Y-%m-%d').str.match(date1_str)]

                            if not filtered_row.empty:
                                leasing_value = filtered_row["租赁财产价值"].values[0]
                                leasing_value /= 1000
                                form_filler = filtered_row["填表人"].values[0]

                                df_first = pd.concat([df_first, pd.DataFrame({"填表人": [form_filler],
                                                                             "租赁时间": [date1],
                                                                             "租赁价值": [leasing_value]})],
                                                     ignore_index=True)

    else:
        for key, values in result.items():
            if "注销融资" not in key:
                converted_values = []
                for sublist in values:
                    converted_sublist = [datetime.datetime.strptime(date_str, '%Y-%m-%d') for date_str in sublist]
                    last_value = converted_sublist[-1]
                    first_value = converted_sublist[0]
                    converted_values.append([last_value])

                for sublist in converted_values:
                    for date_value in sublist:
                        found_match = False
                        for dt in datetime2:
                            if date_value > dt:
                                date3 = date_value.strftime('%Y-%m-%d')
                                found_match = True
                                break

                        if found_match:
                            merged_df["time_new"] = pd.to_datetime(merged_df["time_new"])
                            date3_str = pd.to_datetime(date3).strftime('%Y-%m-%d')
                            first_date = datetime.datetime.strptime(values[0][0], '%Y-%m-%d')
                            date4_str = first_date.strftime('%Y-%m-%d')
                            filtered_row1 = merged_df[merged_df["time_new"].dt.strftime('%Y-%m-%d').str.match(date3_str)]
                            filtered_row2 = merged_df[merged_df["time_new"].dt.strftime('%Y-%m-%d').str.match(date4_str)]
                            if not filtered_row1.empty and not filtered_row2.empty:
                                leasing_value = filtered_row1["租赁财产价值"].values[0]
                                leasing_value /= 1000
                                form_filler = filtered_row1["填表人"].values[0]
                                df_last = pd.concat([df_last, pd.DataFrame({"填表人": [form_filler],
                                                                           "租赁时间": [date4_str],
                                                                           "租赁价值": [leasing_value]})],
                                                   ignore_index=True)

# 美化
df_first = df_first.groupby(["租赁时间", "填表人"], as_index=False)["租赁价值"].sum()
df_last = df_last.groupby(["租赁时间", "填表人"], as_index=False)["租赁价值"].sum()

df_first = df_first[["填表人", "租赁时间", "租赁价值"]]
df_last = df_last[["填表人", "租赁时间", "租赁价值"]]

# 修改日期列的格式为月/日/年，并删除日期和月份前的前导零
df_first["租赁时间"] = pd.to_datetime(df_first["租赁时间"]).dt.strftime("%-m/%-d/%Y")
df_last["租赁时间"] = pd.to_datetime(df_last["租赁时间"]).dt.strftime("%-m/%-d/%Y")

print("仅有初始登记:")
print(df_first)
print("多个登记类型:")
print(df_last)